In [210]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from geopy.geocoders import Nominatim
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, accuracy_score, make_scorer
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [211]:
df = pd.read_csv('SaltyAnimators Salary Collection.csv',encoding='utf-8')
df.describe()


,RATE,Role,Seniority,Location,Years,Company,Gender,Union
count,2302,2301,2229,2281,2281,994,2238,1737
unique,2063,1014,287,587,293,527,4,189
top,60000.00,3D Animator,Junior,Vancouver,3,Freelance,Female,No
freq,8,243,447,218,233,39,1251,1082


In [212]:
print(df.shape)
df.isnull().sum()

(2302, 8)


RATE            0
Role            1
Seniority      73
Location       21
Years          21
Company      1308
Gender         64
Union         565
dtype: int64

In [213]:
#Basic cleanings
for c in df:
    df[c] = df[c].str.lower()
    df[c].str.replace(',','')
    df[c].str.replace(' ','')


In [214]:
#Deal with Gender
tmp = []
for index, row in df.iterrows():
    if 'male' in str(row['Gender']):
        tmp.append(1)
    elif 'female' in str(row['Gender']):
        tmp.append(0)
    else:
        df.drop(index, inplace=True)
df['Gender'] = tmp  

In [215]:
#Deal with Company
tmp = []
for data in df['Company']:
    if 'freelance' in str(data):
        tmp.append(1)
    else:
        tmp.append(0)

df['Company'] = tmp

In [216]:
#Deal with Union
tmp = []
for data in df['Union']:
    if 'no' in str(data):
        tmp.append(1)
    else:
        tmp.append(0)

df['Union'] = tmp

In [217]:
#Deal with Years
tmp = []
for index, row in df.iterrows():
    year = ''.join(i for i in str(row['Years']) if i.isdigit())
    
    if len(year)== 0:
        df.drop(index, inplace=True)
    else:
        tmp.append(year)

df['Years'] = tmp

In [218]:
#Deal with Seniority
tmp = []
for index, row in df.iterrows():
    if 'director' in str(row['Seniority']):
        tmp.append(0)
    elif 'supervisor' in str(row['Seniority']):
        tmp.append(1) 
    elif 'senior' in str(row['Seniority']) or 'sr' in str(row['Seniority']):
        tmp.append(2) 
    elif 'mid' in str(row['Seniority']) or 'junior' in str(row['Seniority']) or 'Intermediate' in str(row['Seniority']) or 'jr' in str(row['Seniority']):
        tmp.append(3)
    elif len(str(row['Seniority']))== 0:
        df.drop(index, inplace=True)
    else:
        df.drop(index, inplace=True)

df['Seniority'] = tmp

In [219]:
#Deal with Rate
tmp = []
for index, row in df.iterrows():
    rate = ''.join(i for i in str(row['RATE']) if i.isdigit())
    if len(rate)>7:
        df.drop(index, inplace=True)
    elif len(rate)== 0:
        df.drop(index, inplace=True)
    else:
        tmp.append(rate)


df['RATE'] = tmp

In [220]:
ohe = OneHotEncoder()
df.dropna(inplace = True)

In [221]:
#Deal with Location
# tmp = []
# loc = Nominatim(user_agent="GetLoc")
# geoLoc = Nominatim(user_agent="GetLoc")

# for index, row in df.iterrows():
#     getLoc = loc.geocode(str(row['Location']))
#     tmp.append(getLoc)

# df['Location'] = tmp
# transformed = ohe.fit_transform(df["Location"].values.reshape(-1, 1))
# df["Location"] = transformed.toarray()
dummy_location = pd.get_dummies(df['Location'])

df = pd.merge(
    left=df,
    right=dummy_location,
    left_index=True,
    right_index=True,
)


In [222]:
#Deal with Role
# transformed = ohe.fit_transform(df["Role"].values.reshape(-1, 1))
# print(transformed)
# df["Role"] = transformed.toarray()
dummy_role = pd.get_dummies(df['Role'])
df = pd.merge(
    left=df,
    right=dummy_role,
    left_index=True,
    right_index=True,
)

df.drop(['Location', 'Role'], axis=1, inplace=True)

In [223]:
df.dropna(thresh=1,inplace = True)
df["RATE"] = df["RATE"].astype("float")
df["Seniority"] = df["Seniority"].astype("float")
df["Years"] = df["Years"].astype("float")
df["Company"] = df["Company"].astype("float")
df["Gender"] = df["Gender"].astype("float")
df["Union"] = df["Union"].astype("float")



y = df['RATE']
X = df.drop(df.columns[[0]], axis=1, inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

regressor = LinearRegression()
regressor.fit(X_train,y_train) 

y_pred = regressor.predict(X_test) 
print(mean_absolute_error(y_pred,y_test))

13991720861218.537


In [224]:
print(y_pred)

[-1.70858949e+12  1.39614562e+05  3.49728500e+05 -1.72980438e+05
 -8.99511875e+04  5.95583569e+13 -1.70858878e+12  4.21656875e+04
 -1.70858712e+12 -1.70858886e+12 -1.39699164e+13 -1.25143773e+13
  7.62299938e+05  3.39915875e+05 -1.20934195e+14  3.99794312e+05
  1.50316338e+06 -3.03731875e+04 -2.43569591e+13  2.32888625e+05
 -1.70858969e+12  1.22161406e+06  1.95637850e+06  2.67079500e+05
  1.33004938e+05 -3.06795750e+05  3.85059375e+05 -1.70858867e+12
 -1.42229664e+13 -4.51101625e+05 -1.24833356e+14  2.67079500e+05
  7.71239375e+04  3.46041625e+05 -8.62704812e+05 -1.70858894e+12
  6.82636163e+13  3.67460000e+05 -1.70858757e+12  1.73328938e+05
  3.15726875e+04 -1.70858865e+12  4.23335625e+05  1.74970974e+13
  1.34165625e+05 -1.25143776e+13  6.46540562e+05  4.03545562e+05
 -2.66943375e+05  7.80899992e+13  1.11273681e+06 -1.70858894e+12
 -1.70858959e+12  3.33714312e+05 -6.18614667e+12 -4.07458534e+13
  2.49697688e+05 -1.70858897e+12 -1.70858896e+12  3.37183750e+04
 -2.28770384e+13 -1.70858

In [225]:
print(y_test)

1759       1650.0
458        3100.0
501       75000.0
679       60000.0
1660      55000.0
1589         93.0
2300       8000.0
557      180000.0
711         470.0
1920        120.0
1011       2200.0
370        4000.0
684          42.0
1135      42000.0
688        7200.0
690       50000.0
319       85800.0
2179      62400.0
1701       2070.0
1950         57.0
1217      50000.0
1474      68000.0
1173      54000.0
331          93.0
1539        130.0
104         400.0
1693        778.0
1336    4500000.0
1764     451800.0
2207      24000.0
142        3250.0
651       74100.0
754       85000.0
2187        950.0
2126      85000.0
1943      98000.0
1559         60.0
605          65.0
1199         80.0
478        2100.0
575        1923.0
818         100.0
110       85000.0
1493        400.0
241       28800.0
2215       1500.0
641       70000.0
479        5500.0
852       90000.0
2063      30000.0
2028       1350.0
719      375078.0
735          48.0
1769    5600000.0
17           90.0
72        